## Урок 1. Алгоритм линейной регрессии. Градиентный спуск

In [1]:
#!pip install bayesian-optimization

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization
import timeit

### Д/З

1. Подберите скорость обучения (alpha) и количество итераций

In [3]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2]])
X

array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  2,  5,  3,  0,  5, 10,  1,  2]])

In [4]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]
y

[45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [5]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2) # <=> 1/n * np.sum((y_pred - y)**2)
    return err

In [6]:
n = X.shape[1]
alpha = 0.055
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')

for i in range(121):
    y_pred = np.dot(W, X)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= alpha * (1/n * 2 * np.sum(X[k] * (y_pred - y)))
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')




Number of objects = 10        
Learning rate = 0.055        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [ 6.94  21.235], MSE = 3047.75
Iteration #10: W_new = [22.06405765 13.06063842], MSE = 848.44
Iteration #20: W_new = [31.21345509  8.7771262 ], MSE = 268.66
Iteration #30: W_new = [36.73513379  6.50786479], MSE = 109.6
Iteration #40: W_new = [40.06112387  5.29211526], MSE = 64.0
Iteration #50: W_new = [42.06149781  4.63338341], MSE = 50.33
Iteration #60: W_new = [43.26313928  4.2724736 ], MSE = 46.05
Iteration #70: W_new = [43.98427548  4.07261073], MSE = 44.67
Iteration #80: W_new = [44.41671151  3.96081125], MSE = 44.21
Iteration #90: W_new = [44.67586386  3.89768846], MSE = 44.05
Iteration #100: W_new = [44.83109194  3.86174724], MSE = 44.0
Iteration #110: W_new = [44.92403348  3.84112851], MSE = 43.98
Iteration #120: W_new = [44.97966334  3.82922172], MSE = 43.97


*2. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

In [7]:
n = X.shape[1]
alpha = 0.055
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')

for i in range(121):
    y_pred = np.dot(W, X)
    err = calc_mse(y, y_pred)
    W -= alpha * (1/n * 2 * np.sum(X * (y_pred - y), axis = 1)) # add axis = 1
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 10        
Learning rate = 0.055        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [ 6.94  21.235], MSE = 3047.75
Iteration #10: W_new = [22.06405765 13.06063842], MSE = 848.44
Iteration #20: W_new = [31.21345509  8.7771262 ], MSE = 268.66
Iteration #30: W_new = [36.73513379  6.50786479], MSE = 109.6
Iteration #40: W_new = [40.06112387  5.29211526], MSE = 64.0
Iteration #50: W_new = [42.06149781  4.63338341], MSE = 50.33
Iteration #60: W_new = [43.26313928  4.2724736 ], MSE = 46.05
Iteration #70: W_new = [43.98427548  4.07261073], MSE = 44.67
Iteration #80: W_new = [44.41671151  3.96081125], MSE = 44.21
Iteration #90: W_new = [44.67586386  3.89768846], MSE = 44.05
Iteration #100: W_new = [44.83109194  3.86174724], MSE = 44.0
Iteration #110: W_new = [44.92403348  3.84112851], MSE = 43.98
Iteration #120: W_new = [44.97966334  3.82922172], MSE = 43.97


### Minimize MSE

In [8]:
def solution(alpha = 0.055, n_iter = 121, shade = 0.01):
    n = X.shape[1]
    W = np.array([1, 0.5])
    for i in range(int(n_iter)):
        y_pred = np.dot(W, X)
        err = calc_mse(y, y_pred)
        W -= alpha * (1/n * 2 * np.sum(X * (y_pred - y), axis = 1))
        alpha  -= alpha * shade
    return -err
    

In [9]:
solution(shade = 0) 

-43.97241493228741

In [10]:
pbounds = {'alpha': (0.01, 0.1), 'n_iter': (121, 1000), 'shade': (0, 0.01)}

optimizer = BayesianOptimization(
    f=solution,
    pbounds=pbounds,
    random_state=1,
    verbose=1
)

In [11]:
start = timeit.default_timer()

optimizer.maximize(
    init_points=2,
    n_iter=100
)

stop = timeit.default_timer()
print('Time: ', stop - start)  

|   iter    |  target   |   alpha   |  n_iter   |   shade   |
-------------------------------------------------------------
|  6        | -43.97    |  0.04757  |  754.2    |  1.196e-0 |
|  18       | -43.97    |  0.04933  |  754.2    |  0.0      |
|  28       | -43.97    |  0.05038  |  754.2    |  0.000854 |
Time:  339.2804967


In [12]:
best_param = list(optimizer.max['params'].values())

In [13]:
solution(*best_param) * -1

43.968749999999986

In [14]:
print('BEST PARAMS:',optimizer.max['params'] )

BEST PARAMS: {'alpha': 0.050381797132125025, 'n_iter': 754.1728044139238, 'shade': 0.0008549036380771859}


In [15]:
43.968749999999986

43.968749999999986